In [3]:
import importlib
import torch
import datafawn
importlib.reload(datafawn)

from pathlib import Path

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device('cpu')
    print("CUDA not available, using CPU")


Using GPU: NVIDIA GeForce RTX 5070 Ti Laptop GPU


In [4]:
# =============== SOUNDSCAPE GENERATOR =============== #
SOUNDSCAPE_CONFIG = {
    'event_sound_map': {
        'front_left_paw_strike': Path('data_example/sounds/22415__anthousai__wind-chimes/398494__anthousai__wind-chimes-single-01.wav'),
        'front_right_paw_strike': Path('data_example/sounds/22415__anthousai__wind-chimes/398493__anthousai__wind-chimes-single-02.wav'),
        'back_left_paw_strike': Path('data_example/sounds/22415__anthousai__wind-chimes/398492__anthousai__wind-chimes-single-03.wav'),
        'back_right_paw_strike': Path('data_example/sounds/22415__anthousai__wind-chimes/398496__anthousai__wind-chimes-single-04.wav')
    }
}
ss_generator = datafawn.SoundScapeFromConfig(soundscape_config=SOUNDSCAPE_CONFIG)

In [5]:
pipeline = datafawn.EventDetectionPipeline(
    soundscape_generators=[ss_generator]
)

In [7]:
results = pipeline.run(
    events_path='data/deerrunning/events/events.json',
    output_dir='data/soundscape_tests/',
    soundscape_input_video='data/deerrunning/pose_estimation/deerrunning_superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2__labeled_before_adapt.mp4'
)

Loading video: data\deerrunning\pose_estimation\deerrunning_superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2__labeled_before_adapt.mp4
Video FPS: 29.97002997002997
Video duration: 20.52 seconds

Processing individual: animal0 (scorer: superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2_)

  Processing back_left_paw_strike:
    Found 16 strikes
    Strike timestamps: ['3.37s', '3.70s', '4.80s']...
    Sound duration: 3.19 seconds
    Created 16 audio clips for back_left_paw_strike

  Processing back_right_paw_strike:
    Found 16 strikes
    Strike timestamps: ['1.50s', '2.37s', '3.30s']...
    Sound duration: 4.04 seconds
    Created 16 audio clips for back_right_paw_strike

  Processing front_left_paw_strike:
    Found 16 strikes
    Strike timestamps: ['1.57s', '2.64s', '4.07s']...
    Sound duration: 5.00 seconds
    Created 16 audio clips for front_left_paw_strike

  Processing front_right_paw_strike:
    Found 17 strikes
    Strike timestamps: ['1.50s', '2.57s', '

MoviePy - Done.
MoviePy - Writing video data\soundscape_tests\soundscapes\SoundScapeFromConfig_output.mp4



MoviePy - Done !
MoviePy - video ready data\soundscape_tests\soundscapes\SoundScapeFromConfig_output.mp4
Done! Output saved to: data\soundscape_tests\soundscapes\SoundScapeFromConfig_output.mp4
📁 Saved soundscape: data\soundscape_tests\soundscapes\SoundScapeFromConfig_output.mp4
